# Similarity Prediction

<br>Subject: INSY-5336 (Python Programming)
<br>Submitted by: 
<br>Name (UTA-ID): Shreyas Jagadeep Shete (1001888859); 
<br>Date: 09 - Dec - 2022
<br>
<br>NOTE: 
<br>1. This is submission to "INSY 5336" course at "The University of Texas at Arlington". 
<br>2. This is property of "Shreyas Jagadeep Shete", no one can use it for any submissions anywhere. 
<br>3. You can use it for the reference and can use it for any project apart from university submissions.

In [1]:
# import libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import urllib.request
# By-pass security
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

all_links = []
# Scraping 500 movie links from 5 pages
for i in range(5):
    req = urllib.request.Request('https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+str(i), headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    hyperlinks = soup.find_all('a', attrs = {'class':'title'})
    all_links.extend(hyperlinks)

In [2]:
# Cleaning the extraction and add proper urls to list
all_urls = []
for i in all_links:
    all_urls.append('https://www.metacritic.com' + i.get('href'))

In [3]:
# Create dictionary with movie name as key and value sub-dictionary with key director name tuple and value is list of genres
dict500 ={}
directorSet = set()
for url in all_urls:
    subDict ={}
    req = urllib.request.Request(url, headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    title = soup.find_all('div', attrs = {'class':'product_page_title oswald'})
    movieName = title[0].get_text().strip().replace('\n', ' ')
    director=[]
    director_div = soup.find('div', attrs = {'class':'director'})
    director_link = director_div.find_all('a')
    for i in director_link:
        director.append(i.get_text())
        directorSet.add(i.get_text().lower())
    genres_div = soup.find('div', attrs = {'class':'genres'})
    genres_list = genres_div.find_all('span')
    genres = []
    for i in genres_list:
        genres.append(i.get_text())
    subDict[tuple(director)] = genres[2:]
    dict500[movieName] = subDict

In [4]:
# import csv to store the dictionary to csv file
import csv
fhandle = open('movies_myname.csv', mode='w', newline = '') #line 1
movie_writer = csv.writer(fhandle) #line 2
movie_writer.writerow(['Movie_Name', 'Director(s)', 'Genre(s)'])
for key1, value1 in dict500.items():
    for key2, value2 in value1.items():            
        movie_writer.writerow([key1, ', '.join(list(key2)), ', '.join(value2)])
fhandle.close()

In [5]:
# Ask for User input and based on the options perform respective operation
while True:
    user_option = input("What do you want to check on Metacritic? (Please choose ‘movie’, ‘director’, or ‘comparison’) ")
    if user_option.lower() in ['movie', 'director', 'comparison']:
        break
    print("Wrong input, please try again")
# If input is Movie Name, Output the director name and its genre
if user_option.strip().lower() == 'movie':
    movie_ToCheck = input('What movie do you want to check? ')
    flag = False
    for key in dict500.keys():
        if movie_ToCheck.lower() in key.lower():
            subDict = dict500[key]
            for director, genre in subDict.items():
                directors = " ".join(list(director))
                genreList = ", ".join(genre)
                print("The movie is ", key)
                print("The director of the movie is ", directors)
                print("The genre of the movie is ", genreList)
                flag = True
    if not flag:
        print("Movie is not available, please try again by running the cell.")
        
                
# If input is Director Name, Output his directed movie and his most directed genres
elif user_option.strip().lower() == 'director':
    while True:
        director = input('Who do you want to check? ').strip()
        if director.lower() not in directorSet:
            print("The director is not available, Please try again.")
            continue
        break
    movieList = []
    genreDict = {}
    for key, val in dict500.items():
        directors = list(list(val.keys())[0])
        for i in directors:
            if director.lower() == i.lower():
                movieList.append(key)
                for j in list(val.values())[0]:
                    if j in genreDict:
                        genreDict[j] +=1
                    else:
                        genreDict[j] = 1
    print(f"{director} has directed ",", ".join(movieList))
    genreList = []
    for i ,j in genreDict.items():
        genreList.append(f"{i}:{j}")
    print("His/ Her most directed genres are ",", ".join(genreList))

    
# If input is Comparison, Ask for the two director names to compare and Output their cosine Similarity    
elif user_option.strip().lower() == 'comparison':
    while True:
        director1 = input("Enter first director name: ").strip()
        if director1.lower() not in directorSet:
            print("Director is not available please try again.")
            continue
        break
    while True:
        director2 = input("Enter second director name: ").strip()
        if director2.lower() not in directorSet:
            print("Director is not available please try again.")
            continue
        break
    
    print('\n***************************************************\n')
    
    movie_directed1 = []
    genres_1 = []
    for key1, value1 in dict500.items():
        for key2, value2 in value1.items():
            if director1 in key2:
                genres_1.extend(value2)
                movie_directed1.append(key1)
    
    movie_directed2 = []
    genres_2 = []
    for key1, value1 in dict500.items():
        for key2, value2 in value1.items():
            if director2 in key2:
                genres_2.extend(value2)
                movie_directed2.append(key1)
                
    common_genres = set(genres_1).union(set(genres_2))
    dict1 = {}
    for i in genres_1:
        if i not in dict1:
            dict1[i] = 1
        else:
            dict1[i] = dict1[i] + 1
    dict2 = {}
    for i in genres_2:
        if i not in dict2:
            dict2[i] = 1
        else:
            dict2[i] = dict2[i] + 1
    
    dict_common = {}
    for i in common_genres:
        dict_common[i] = 0
        
    dict_common.update(dict1)
    
    dict_common2 = {}
    for i in common_genres:
        dict_common2[i] = 0
    dict_common2.update(dict2)

    x = list(dict_common.values())
    y = list(dict_common2.values())
    
    numerator = 0
    for i in range(len(x)):
        numerator = numerator + x[i]*y[i]
        
    sum_x = 0
    for i in x:
        sum_x = sum_x + i**2
        
    sq_x = sum_x**0.5
    
    sum_y = 0
    for i in y:
        sum_y = sum_y + i**2
    
    sq_y =sum_y**0.5
    
    denominator = sq_x*sq_y
    similarity = numerator/denominator
    
    print(f"{director1} has directed ",", ".join(movie_directed1))
    print('-----------------------------------------------------')
    print("His/ Her most directed genres are ",", ".join(genres_1))
    print('\n')
    print(f"{director2} has directed ",", ".join(movie_directed2))
    print('-----------------------------------------------------')
    print("His/ Her most directed genres are ",", ".join(genres_2))
    print('\n\n')

    print(f'Based on that, they have a cosine similarity score of {similarity}')
    print('\n\nIn conclusion we say that:\n')
    print(f'Cosine similarity between {director1} and {director2} is {similarity}')
    print('-----------------------------------------------------')
    print(f'{director1} and {director2} are {round(similarity*100, 3)}% similar to each other with respect to the genre that they have directed.')
    print('-----------------------------------------------------\n\n\n\nBye\nThank-you!')
    
# Error out if wrong option is entered
else:
    print("ERROR! You entered wrong option, Restart and Try again!")

What do you want to check on Metacritic? (Please choose ‘movie’, ‘director’, or ‘comparison’) Comparison
Enter first director name: asdkjhksdhf
Director is not available please try again.
Enter first director name: Francis Ford Coppola
Enter second director name: hgaddsjkgfasdkjghfjkasdghfjkads
Director is not available please try again.
Enter second director name: Orson Welles

***************************************************

Francis Ford Coppola has directed  The Godfather 1972, Apocalypse Now 1979, Apocalypse Now: Final Cut 2019, Apocalypse Now Redux 2001, The Godfather: Part II 1974, The Conversation 1974
-----------------------------------------------------
His/ Her most directed genres are  Drama, Thriller, Crime, Action, Drama, War, Drama, Mystery, War, Action, Drama, War, Drama, Thriller, Crime, Drama, Mystery, Thriller


Orson Welles has directed  Citizen Kane 1941, Touch of Evil 1958, Chimes at Midnight 1967, The Magnificent Ambersons 1942, F for Fake 1975
---------------